In [1]:
# scrape trump transcripts from https://www.rev.com

In [2]:
import pandas as pd
import time
import re
import requests
from datetime import datetime
from bs4 import BeautifulSoup as bs

In [3]:
all_briefings_resp = requests.get("https://www.rev.com/blog/transcript-tag/trump-coronavirus-task-force-briefing-transcripts")
all_briefings = bs(all_briefings_resp.text)

first_page_briefings = bs(all_briefings_resp.text)

first_page_briefings_list = first_page_briefings.find_all("div",{"class":"fl-post-column"})

current_page_briefings = first_page_briefings
# check next page

#check if there's a 'next' button and store associated url
next_url = current_page_briefings.find("a",{"next page-numbers"})['href']

# if so, collect the subsequent urls to scrape
if next_url:
    page_url_list = [all_briefings_resp.url]
    while next_url:
        page_url_list.append(next_url)
        current_page_briefings = bs(requests.get(next_url).text)
        time.sleep(.4)
        try:
            next_url = current_page_briefings.find("a",{"next page-numbers"})['href']
        except:
            next_url = None
            
transcript_urls = []
for url in page_url_list:
    page_briefings = bs(requests.get(url).text)
    page_briefings_list = page_briefings.find_all("div",{"class":"fl-post-column"})
    time.sleep(.5)
    for p in page_briefings_list:
        transcript_urls.append(p.find("a")['href'])

In [4]:
all_briefing_dfs = []

for url in transcript_urls:
    briefing = bs(requests.get(url).text)
    time.sleep(.4)
    
    briefing_text = briefing.find("div",{"class":"fl-callout-text"})
    
    # empty lists to store transcript columns
    speaker = []
    timestamp = []
    text = []

    # iterate over transcript paragraphs
    for paragraph in briefing_text.find_all("p"):

        name_match = re.search('(.+?):', paragraph.text)
        if name_match:
            speaker.append(name_match.group(1))

        timestamp_match = re.search('\((.+?)\)', paragraph.text)
        if timestamp_match:
            timestamp.append(timestamp_match.group(1))

        corp_match_index = paragraph.text.find(")")+2
        if corp_match_index:
            text.append(paragraph.text[corp_match_index:])

    # combine lists into a single df
    briefing_data = pd.DataFrame(list(zip(timestamp, speaker, text)),
                                     columns = ['timestamp','speaker','text'])

    # extract briefing date and add to df
    date_string = briefing.find_all("div",{"class":"fl-rich-text"})[0].text
    if date_string:
        briefing_date = datetime.strptime(date_string, '%b %d, %Y')

    briefing_data.insert(0,'date',briefing_date)
    
    # export single briefing as csv
    filename = "../data/whtfb/" + briefing_date.strftime("%Y-%m-%d")+("_tfb.csv")
    briefing_data.to_csv(filename,index=False)   
    
    # store all briefings in a single df
    all_briefing_dfs.append(briefing_data)

In [5]:
merged_briefings = pd.concat(reversed(all_briefing_dfs))
merged_briefings.to_csv("../data/whtfb/all_briefings.csv",index=False)